# Домашнее задание №2
## 1 - 3(c) К полученной в ходе первого задания БД добавлена таблица платежей по контрактам. Код запрашивает из БД таблицу платежей по контрактам и по ней находит моменты просрочки 90+ для каждого контракта. Предлагается задать горизонт риска и по выбранному горизонту строится таблица заемщиков с кредитами сроком не меньше горизонта риска с флагом {дефолт/недефолт} на выбранном горизонте с момента выдачи кредита  Вся информация о моментах дефолтов возвращается обратно в базу данных в виде отдельной таблицы.

In [1]:
import sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#соединяемся с созданной в задании 1 базой данных data_base
conn = sqlite3.connect("data_base.db")

In [3]:
#распаковываем таблицу платежей из excel файла и загружаем в базу данных
payments_table = pd.read_excel('payments.xls')

In [4]:
#создаем пустую заготовку в базе данных для размещения платежей
cursor = conn.cursor()
cursor.execute("""CREATE TABLE Payments (
id_number INT,
contract_number INT,
date DATE,
amount_due INT,
amount_paid INT, 
PRIMARY KEY (id_number),
FOREIGN KEY (id_number) REFERENCES Contracts(id_number)) """)

conn.commit()

In [5]:
#наполняем таблицу Payments данными по платежам
cursor = conn.cursor()
for i in range(0, payments_table.shape[0]):
    cursor.execute(""" INSERT INTO payments 
    VALUES 
    (%d, %d, '%s', %d, %d)""" % (
        i,
        payments_table['Contract Number'][i],
        payments_table['Date'][i],
        payments_table['Amount Due'][i], 
        payments_table['Amount Paid'][i] 
        ))

conn.commit()

In [6]:
cursor = conn.cursor()

payments_table = pd.read_sql_query("SELECT * FROM payments", conn)
payments_table.keys()

Index(['id_number', 'contract_number', 'date', 'amount_due', 'amount_paid'], dtype='object')

In [7]:
s_dates = payments_table.groupby(['contract_number'], as_index=False).agg({'date': 'min'})
s_dates['amount_due'] = np.nan
s_dates['amount_paid'] = np.nan

for ind in s_dates.index:
    row = s_dates.loc[ind]
    t_df = payments_table[(payments_table.contract_number == row.contract_number) & (payments_table.date == row.date)]
    s_dates.loc[ind, 'amount_due'] = t_df.amount_due.iloc[0]
    s_dates.loc[ind, 'amount_paid'] = t_df.amount_paid.iloc[0]
if np.any(s_dates.amount_paid - s_dates.amount_due < 0):
    mb_zero_default = s_dates[s_dates.amount_paid - s_dates.amount_due < 0].contract_number.unique()
    print("Проверьте дефолтные контракты:", mb_zero_default) #дефолтные означает, что просрочка существует с самого начала (с нулевого периода)

In [8]:
#Равенство единице означает, что в контаркте не было просрочки, а 0 что должен был быть платеж, но его нет
payments_table['overdue_days'] = 1
for contract in payments_table.contract_number.unique():
    temp_df = payments_table[payments_table.contract_number == contract].sort_values(['date'])
    index_arr = temp_df.index.tolist()
    for i in range(1, len(index_arr)):
        prev_ind = index_arr[i - 1]
        ind = index_arr[i]
        if payments_table.loc[ind, 'amount_paid'] < payments_table.loc[ind, 'amount_due']:  
            if payments_table.loc[prev_ind, 'overdue_days'] == 1:  # Не было просрочки
                payments_table.loc[ind, 'overdue_days'] = 0  # Есть просрочка
            else:
                new_over_days = 30 #вычисляет количество днем просрочки как 1 месяц = 30 дней
                payments_table.loc[ind, 'overdue_days'] = payments_table.loc[prev_ind, 'overdue_days'] + new_over_days  
        elif payments_table.loc[ind, 'amount_paid'] > payments_table.loc[ind, 'amount_due']:
            months_covered = int(round(payments_table.loc[ind, 'amount_paid'] / payments_table.loc[ind, 'amount_due'], 0))  
            months_covered = months_covered - 1  
            payments_table.loc[ind, 'overdue_days'] = payments_table.loc[prev_ind, 'overdue_days'] - months_covered * 30
            if payments_table.loc[ind, 'overdue_days'] < 0:
                payments_table.loc[ind, 'overdue_days'] = 1  # Если долг погашен - 0 - просрочки нет 

In [9]:
#образуем таблицу, выводящую дефолты с номерами контракта
default_dates = payments_table[payments_table.overdue_days == 90].groupby(['contract_number'], as_index=False).agg({"date": "max"})
default_dates = default_dates.rename({"contract_number": "Contract Number", "date": "Default Date"}, axis=1)
default_dates['Default Date'] = [x[:-9] for x in default_dates['Default Date']] #в таблице дат дефолтов не будет секунд
default_dates

,Contract Number,Default Date
0,100052,2017-12-30
1,100375,2017-01-02
2,100668,2018-06-29
3,100684,2017-12-14
4,100875,2017-03-05
5,100901,2016-11-03


In [10]:
#отдельно добавим таблицу дефолтов в базу данных 
cursor = conn.cursor()
cursor.execute("""CREATE TABLE Defaults (
contract_number INT,
default_date DATE,
PRIMARY KEY(contract_number)) """)

conn.commit()

In [11]:
#загрузим данными базу данных по дефолтам
for i in range(0, default_dates.shape[0]):
    cursor.execute("""INSERT INTO Defaults 
    VALUES
    (%d, '%s')"""% (
    default_dates['Contract Number'][i],
    default_dates['Default Date'][i]))
    
conn.commit()

In [12]:
#На следующем этапе необходимо задать горизонт риска (считать в месяцах с начала заключения контракта для удобства). 
#Возьмем горизонт риска = 20, то есть 20 месяцев. Далее можете на этом этапе установить необходимый Вам горизонт риска. 
print ('Горизонт риска =') 
ask_risk_horizon = input()

Горизонт риска =
20


In [13]:
risk_horizon = int(ask_risk_horizon)
risk_horizon

20

In [14]:
#Создадим переменную, в которую будут добавляться данные типа (номер контракта, дата дефолта, флаг дефолт/недефолт)
default_risk_horizon = []

contract_number = list(payments_table['contract_number'])
contract_number_unique = list(set(contract_number))

for i in contract_number_unique:
    table = payments_table[payments_table['contract_number'] == i]
    if table.shape[0] >= risk_horizon:
        table = table[:risk_horizon]
        table = table.reset_index()
        for j in range(0, table.shape[0]-3):
            if (table['amount_paid'][j] < table['amount_due'][j] and table['amount_paid'][j+1] < table['amount_due'][j+1] and
                table['amount_paid'][j+2] < table['amount_due'][j+2] and table['amount_paid'][j+3] < table['amount_due'][j+3]):
                default_date = table['date'][j + 3]
                default_date = (str(pd.to_datetime(default_date).year) + '-' + str(pd.to_datetime(default_date).month) + 
                                '-' + str(pd.to_datetime(default_date).day))
                while (i, '0000-00-00', 0) in default_risk_horizon: #Здесь мы лезим в общий список просрочек и контрактов, просто фиктивные переменные 0 и 1 не подойдут
                                default_risk_horizon.remove ((i, '0000-00-00', 0)) #0000-00-00 для удобства обозначения год-месяц-день при отсутвии дефолта, далее дроп
                default_risk_horizon.append((i, default_date, 1)) 
                break 
            else: default_risk_horizon.append((i, '0000-00-00', 0))

In [15]:
#объединим контракты, чтобы на выходе таблица содержала в себе данные типа: номер контракта(уникальный),
#дата просрочки(или же 0000-00-00 если ее нет) и флаг дефолта(фиктивная перменная 0 или 1)
default_risk_horizon = list(set(default_risk_horizon))
print (len(default_risk_horizon))
default_risk_horizon

42


[(100585, '0000-00-00', 0),
 (100368, '0000-00-00', 0),
 (101028, '0000-00-00', 0),
 (100737, '0000-00-00', 0),
 (100206, '0000-00-00', 0),
 (100520, '0000-00-00', 0),
 (100703, '0000-00-00', 0),
 (100540, '0000-00-00', 0),
 (100323, '0000-00-00', 0),
 (100933, '0000-00-00', 0),
 (100297, '0000-00-00', 0),
 (100022, '0000-00-00', 0),
 (100600, '0000-00-00', 0),
 (100872, '0000-00-00', 0),
 (100380, '0000-00-00', 0),
 (100684, '2017-12-14', 1),
 (100113, '0000-00-00', 0),
 (100990, '0000-00-00', 0),
 (100847, '0000-00-00', 0),
 (100176, '0000-00-00', 0),
 (100797, '0000-00-00', 0),
 (100122, '0000-00-00', 0),
 (100743, '0000-00-00', 0),
 (100068, '0000-00-00', 0),
 (100929, '0000-00-00', 0),
 (100127, '0000-00-00', 0),
 (100476, '0000-00-00', 0),
 (100441, '0000-00-00', 0),
 (100190, '0000-00-00', 0),
 (100958, '0000-00-00', 0),
 (100869, '0000-00-00', 0),
 (100997, '0000-00-00', 0),
 (100431, '0000-00-00', 0),
 (100524, '0000-00-00', 0),
 (100303, '0000-00-00', 0),
 (100765, '0000-00-0

In [16]:
#соединяемся с созданной базой данных data_base
#создаем пустую заготовку в базе данных для размещения данных по дефолтам на горизоне риска
conn = sqlite3.connect("data_base.db")
cursor = conn.cursor()
cursor.execute("""CREATE TABLE Risk_horizon20 (
contract_number INT,
default_date DATE,
default_flag BOOLEAN)""".format(risk_horizon))

conn.commit()

In [17]:
#заполняем таблицу готовыми данными из таблицы default_risk_horizon
cursor = conn.cursor()
for i in default_risk_horizon:
    cursor.execute("""INSERT INTO Risk_horizon20 
    VALUES
    (%d, '%s', %d)""".format(risk_horizon) % (
    i[0],
    i[1],
    i[2]))
    
conn.commit()

#### итого: в уже имеющуюся базу данных добавлены 2 таблицы: defaults , отражающие все дефолты, а также таблица risk_horizon, в которой содержатся моменты дефолтов на горизонте риска 20 месяцев.